In [1]:
import sys
sys.path.append('unet_backbones')
from unet_backbones.backbones_unet.model.unet import Unet
from unet_backbones.backbones_unet.utils.dataset import SemanticSegmentationDataset
from unet_backbones.backbones_unet.model.losses import DiceLoss
from unet_backbones.backbones_unet.utils.trainer import Trainer
from unet_backbones.backbones_unet.utils.reproducibility import set_seed
from unet_backbones.datasets import VOCSegmentation
from unet_backbones.utils import ext_transforms as et

import os
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import Normalize
from torch.utils.data import DataLoader
from unet_backbones.metrics import StreamSegMetrics

import json
import numpy as np

In [2]:
seed = 42
voc_data_root = "datasets/VOC2012"
voc_crop_size = 256
voc_num_classes = 21
small_decoder = True

save = "experiments"
dataset = "pascalvoc2012"
encoder = "convnext_tiny"
epochs = 150
lr = 1e-4

save_path = os.path.join(save, dataset, encoder, "small_deocder_" + str(small_decoder), "epochs_" + str(epochs), "lr_"+str(lr), 'seed_'+str(seed))
model_path = os.path.join(save_path , "model")
json_path = os.path.join(save_path, "losses.json")
os.makedirs(save_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

In [3]:
def get_logger(save_folder):
    log_path = str(save_folder) + '/log.log'
    logging.basicConfig(filename=log_path, filemode='a')
    logger_name = "main-logger"
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)
    handler = logging.StreamHandler()
    fmt = "[%(asctime)s %(levelname)s %(filename)s line %(lineno)d %(process)d] %(message)s"
    handler.setFormatter(logging.Formatter(fmt))
    logger.addHandler(handler)
    return logger

In [4]:
logger = get_logger(save_path)

In [5]:
set_seed(seed)

> SEEDING DONE


In [6]:
val_transform = et.ExtCompose([
                et.ExtResize(voc_crop_size),
                et.ExtCenterCrop(voc_crop_size),
                et.ExtToTensor(),
            ])

train_dst = VOCSegmentation(root = voc_data_root, 
                            year = "2012", 
                            image_set = 'train', 
                            download = True, 
                            transform = val_transform)

val_dst = VOCSegmentation(root = voc_data_root, 
                        year = "2012",
                        image_set='val', 
                        download = True, 
                        transform = val_transform)

Using downloaded and verified file: datasets/VOC2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: datasets/VOC2012/VOCtrainval_11-May-2012.tar


In [7]:
train_loader = DataLoader(train_dst, batch_size=2, num_workers=1, shuffle=True)
val_loader = DataLoader(val_dst, batch_size=4, num_workers=1, shuffle=True)

In [8]:
model = Unet(
    backbone = encoder,
    small_decoder = small_decoder,
    in_channels = 3,
    num_classes = voc_num_classes,
)

print(model)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("\n\n\n\t\t\tNUMBER OF PARAMETERS: {}".format(num_params))

Unet(
  (encoder): FeatureListNet(
    (stem_0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (stem_1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    (stages_0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
 

In [9]:
class Args:
    def __init__(self, epsilon, mode, alpha, iterations, attack, num_classes, norm, targeted):
        self.epsilon = epsilon                 # epsilon for adversarial attack
        self.mode = mode                       # choices={'adv_attack', 'adv_train', 'train', 'test'}
        self.alpha = alpha                     # alpha for adversarial attack
        self.iterations = iterations           # number of iterations for adversarial attack
        self.attack = attack                   # choices={'fgsm', 'cospgd', 'segpgd', 'pgd'}
        self.num_classes = num_classes
        self.norm = norm                       # choices={'inf', 'two', 'one'}
        self.targeted = targeted               # choices={'False', 'True'}

In [10]:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

model = nn.Sequential(Normalize(mean = mean, std = std), model)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=lr)
criterion = {"cross_entropy": nn.CrossEntropyLoss(ignore_index=255), "dice_loss": DiceLoss()} 
metrics = StreamSegMetrics(voc_num_classes)

args = Args(0.03, "train", 0.01, 1, "fgsm", voc_num_classes, "inf", "True")

In [11]:
trainer = Trainer(
    model,                    # UNet model with pretrained backbone
    criterion = criterion["cross_entropy"],
    optimizer = optimizer,
    epochs = epochs,
    metrics = metrics,
    args = args,
    logger = logger,
    model_save_path = os.path.join(model_path, "best_model.pt")
)

/root/autodl-tmp/my-cospgd/unet_backbones/backbones_unet/utils/trainer.py:314: UserWarning: Device was automatically selected: cuda
  warnings.warn(msg)


In [12]:
trainer.fit(train_loader, val_loader)

torch.save(
    {
        "epoch": epochs, 
        "model_state_dict": trainer.model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "train_loss": trainer.train_losses_,
        "val_loss": trainer.val_losses_
    }, 
    os.path.join(model_path, "final_model.pt")
)

losses = {
    "train loss": trainer.train_losses_.detach().cpu().tolist(), 
    "val loss": trainer.val_losses_.detach().cpu().tolist(), 
    "score": trainer.metrics.get_results()
}

json_losses = json.dumps(losses, indent=4)
with open(json_path, "w") as f:
    f.write(json_losses)

Traning Model on 150 epochs:   0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/732 [00:00<?, ? training-batch/s]

  0%|          | 0/732 [00:00<?, ? training-batch/s]

  0%|          | 0/732 [00:00<?, ? training-batch/s]

  0%|          | 0/732 [00:00<?, ? training-batch/s]

...

  0%|          | 0/732 [00:00<?, ? training-batch/s]

  0%|          | 0/732 [00:00<?, ? training-batch/s]

  0%|          | 0/732 [00:00<?, ? training-batch/s]

  0%|          | 0/363 [00:00<?, ? validating-batch/s]

[2024-11-08 01:40:49,127 INFO trainer.py line 140 16973] epoch: 150   Overall Acc: 0.8918597289105804    Mean Acc: 0.703029147690643    FreqW Acc: 0.8174419488041585    Mean IoU: 0.5817612342520571    Class IoU: {0: 0.9068912917818385, 1: 0.7773895335135785, 2: 0.4634779149022809, 3: 0.6013393821942414, 4: 0.49211931828269734, 5: 0.4736582741066393, 6: 0.7900323181294404, 7: 0.7673313436926165, 8: 0.6156565062517468, 9: 0.23157538217037602, 10: 0.4110849714732751, 11: 0.46911281346968886, 12: 0.5586858416231596, 13: 0.5108706598542426, 14: 0.6796131910091536, 15: 0.7865966836355035, 16: 0.44609764310717087, 17: 0.6269254169249052, 18: 0.346313446827237, 19: 0.7032397227965209, 20: 0.5589742635468843}    
